In [2]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

In [3]:
tf.__version__

'2.11.0'

In [4]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [5]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


2024-03-08 10:34:58.300520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 10:34:58.305484: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [7]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [8]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [67]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(5*4*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((5, 4, 256)))

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))

    # Flatten the last dimension to remove the extra dimension
    model.add(layers.Lambda(lambda x: tf.image.resize(x, (25, 20))))
    model.add(layers.Lambda(lambda x: tf.squeeze(x, axis=-1)))


    # Convert the largest element in each row to 1 and others to 0
    model.add(layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=20)))

    return model
    
    # model = Sequential([
    #     layers.Input(shape=(latent_dim,)),
    #     layers.Dense(128, activation='relu'),
    #     layers.Dense(256, activation='relu'),
    #     layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
    #     layers.Reshape(output_shape),
    #     layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
    #     layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    # ])

    # return model

In [68]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator(latent_dim, output_shape)

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

TypeError: build_generator() takes 0 positional arguments but 2 were given

In [55]:
discriminator = build_discriminator()
# discriminator.load_weights('./acp_mhcnn_weights.h5')

In [56]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 311ms/step


array([[0.46569976, 0.5343002 ],
       [0.45725295, 0.5427471 ],
       [0.4835595 , 0.5164406 ],
       [0.47129807, 0.52870184],
       [0.46295658, 0.53704345]], dtype=float32)

In [57]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

1/1 [==============================] - 0s 199ms/step
[[0.50113446 0.49886557]
 [0.4668543  0.53314567]
 [0.36179316 0.6382069 ]
 [0.4142299  0.5857701 ]
 [0.4856474  0.51435256]]


In [58]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [93]:
def generator_loss(fake_output):
    print('fake output')
    print(fake_output)
    print('ones like output')
    print(tf.ones_like(fake_output))
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

In [70]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [71]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 88ms/step
[[0.39160654 0.60839343]
 [0.47673628 0.5232637 ]
 [0.35599425 0.6440057 ]
 [0.45443636 0.5455637 ]
 [0.47972736 0.5202726 ]]
[[0.46569976 0.5343002 ]
 [0.45725295 0.5427471 ]
 [0.4835595  0.5164406 ]
 [0.47129807 0.52870184]
 [0.46295658 0.53704345]]
tf.Tensor(70.78352, shape=(), dtype=float32)
tf.Tensor(1.4031761, shape=(), dtype=float32)


In [72]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [73]:
def generate_and_save_images(model, epoch, test_input):
    pass

In [74]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [81]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      print(real_output)
      print(fake_output)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)


    i = 0
    for gradient in gradients_of_generator:
      if gradient == None:
        gradients_of_generator[i] = 0.5
      i += 1
    print(gradients_of_generator)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Produce images for the GIF as you go
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
train(train_dataset, EPOCHS)

In [106]:
noise = tf.random.normal([5, 100])

peptides = X1[:5, :15, :]
with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_bpf = generator(noise, training=True)

    real_output = discriminator(peptides, training=True)
    fake_output = discriminator(generated_bpf[:,:15,:], training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    
    print(gen_loss)
    print(disc_loss)

gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
print(gradients_of_generator)

fake output
tf.Tensor(
[[0.11263568 0.8873643 ]
 [0.98583615 0.0141638 ]
 [0.91725624 0.0827438 ]
 [0.34601218 0.6539878 ]
 [0.00347824 0.99652183]], shape=(5, 2), dtype=float32)
ones like output
tf.Tensor(
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]], shape=(5, 2), dtype=float32)
tf.Tensor(1.1377039, shape=(), dtype=float32)
tf.Tensor(1.7026567, shape=(), dtype=float32)
[None, None, None, None, None, None, None, None, None, None]
